# Conditional Choice Probabilitiy Estimators in 5 Easy Steps!

## Author: Eric Schulman

The following guide demonstrates how to use conditional choice probability estimators in Python. It was written in part as a homework for the University of Texas second year course in industrial organization. These estimators are the normal way to think about the future's influences decisions in industrial organization and related fields.

To demonstrate how to use (and implement) a CCP estimator, we recover parameters for the cost function in [Rust 1987](https://www.jstor.org/stable/1911259). Rust's paper considers the decision of a bus manager. The bus manager decides whether or not to replace bus engine for his fleet of buses in Madison, Wisconsion. Replacing the engine has a high cost in the present, but letting the engine accumulate mileage makes the bus more likely to break down in the future. Our goal is estimating parameters that tell us the importance of mileage when the bus manager decides to replace the engines.

The bus manager's problem is very general and has become the 'mascot' for dynamic decisions in industrial organization. Many macroeconomics textbook excersizes call it a 'tree-cutting' problem. The bus engine replacement problem has The manager has a very simple 'yes' or 'no' decision. However, his 'yes' or 'no' depends on the future. To estimate the importance of mileage in the bus manager's decision, we must calculate a value function. A much simple method to predicting the bus manager's decision using mileage would be a logit. Why introduce a value function? Well, Rust wanted to predict agents behavior. Rust demonstrated that adding a structural 'assumption', that agent's discount factor $\beta >0$, does a better job predicitng the probability of replacment. 


Rust's called his approach to predicting agent's choices in this situation the nested fixed point algorithm (NFXP). To think about the future, he wrote a Bellman equation that contained the expected value of 'yes' and 'no' based on the current mileage. He alternated between calculating the fixed point of the Bellman Operator and finding parameters that told him the value of the mileage and using a maximum likelihood routine. Rust has code for solving this value function and estimate parameters in Gauss on his [website](https://editorialexpress.com/jrust/nfxp.html). To find the fixed point of the Bellman operator in Python you can find code [here](https://nbviewer.jupyter.org/github/QuantEcon/QuantEcon.notebooks/blob/master/ddp_ex_rust96_py.ipynb).

The more recent approach to predicting dynamic choices is called conditional choice probability (CCP) estimation. This approach works similar to Rust's assymptotically. However, it simplifies Rust's NFXP. Instead of embedding a value function into a MLE routine every time, you start with a simple estimate of the choice probabilities and adjust this estimate to account for the future. Using the choice probabilities, you can calculate an estimate of the value function. This value function will help you adjust your estimates of how mileage influences the replacement probability. This approach was first introduced to the literature in [Hotz Miller 1993](https://www.jstor.org/stable/2298122). The code and data I used for this guide from comes from Victor Aguirregabiria and Pedro Mira's website accompanying their paper [Aguirregabiria Mira 2002](http://individual.utoronto.ca/vaguirre/wpapers/program_code_survey_joe_2008.html) (more on them later).


## Step 1: Pre-processing the data and structural constants

In this step 2 important things happen:

1. We set the discount factor. The most important aspect of CCP is setting your discount factor. Implicitly, our choice is an assumption about the bus manager because nothing in the data tells us about agent's discount factor (for more about this see [Magnac Thesmar 2002](https://www.jstor.org/stable/2692293)). All we see are mileage and replacment decisions. This assumption better fits the data.

2. We discretize our continous data on mileage so we can think about the bus manager's problem as a Markov Decision process to calculate our Bellman equation.

In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats #for kernel function

In [33]:
#fix the bus .dat from augirregabiria and Mira's website
data = np.fromfile('bus1234.dat')
data = data.reshape(len(data)/6,6)
data = pd.DataFrame(data,columns=['id','group','year','month','replace','miles'])

#save to .csv so other people don't need to be confused
data.to_csv("bus1234.csv")

#divide by 1e6 (use the same scale are Rust and AM)
data['miles'] = (data['miles'])/1e6

#switch to date time for ease 
data['date'] = pd.to_datetime(data[['year', 'month']].assign(Day=1))
data = data[['id','group','date','replace','miles']]

#lag date
date_lag = data.copy()
date_lag['date'] = date_lag['date'] - pd.DateOffset(months=1)
data = data.merge(date_lag, how='left', on=['id','group','date'] , suffixes=('','_next'))
data = data.dropna()

print data.max()

id                              162
group                        530875
date            1985-04-01 00:00:00
replace                           1
miles                      0.388254
replace_next                      1
miles_next                 0.388254
dtype: object


In [19]:
#constants
BETA = .9999
GAMMA = .5772 #euler's constant

#size of step in discretization
STEP = .002

#make states global variables
STATES = np.arange(data['miles'].min(),data['miles'].max() + STEP, STEP)

## Step 2: Calculating Choice Probabilities 'Non-Parametrically'

To do CCP estimation, we must estimate probability of 'yes' and 'no' using very simple methods.

This requires naively estimating:
1. The amount that each bus's mileage $x$ will increase depending on the engine replacement decision $i$
2. The probability of engine replacement decision $i$ and 'conditional on mileage $x$.

I estimated these probabilities using the same approach as Aguirregabiria and Mira. However, you can experiment with other (consistent) methods.

In [12]:
def miles_pdf(i_obs, x_obs, x_next):
    """estimation of mileage pdf following AM using the
    kernel function
    
    this corresponds to pdfdx in AM's code"""
    
    #figure out max number of steps
    dx = (1-i_obs)*(x_next - x_obs) + i_obs*x_next
    
    #number of 'transition' states
    dx_states = np.arange(dx.min(),dx.max() +STEP , STEP)
    
    #use kernel groups to make pdf
    kernel1 = stats.gaussian_kde(dx, bw_method='silverman')
    pdfdx = kernel1(dx_states)
    
    return np.array([pdfdx/pdfdx.sum()]).transpose()


MILES_PDF = miles_pdf(data['replace'], data['miles'], data['miles_next'])

In [13]:
def transition_1(i_obs, x_obs , x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat2 in AM's code"""
    
    #transitions when i=1
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #zero probability of transitioning to large states
    zeros = np.zeros( (len(STATES),len(STATES)-pdfdx.shape[1]) )
    
    #transitioning to first state and 'jumping' dx states
    fmat1 = np.tile(pdfdx,(len(STATES),1))
    fmat1 = np.concatenate( (fmat1, zeros), axis=1 )

    return fmat1

FMAT1 = transition_1(data['replace'], data['miles'],data['miles_next'])

In [14]:
def transition_0(i_obs, x_obs, x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat1 in AM's code"""
    
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #initialize fmat array, transitions when i=0
    end_zeros = np.zeros((1, len(STATES) - pdfdx.shape[1]))
    fmat0 = np.concatenate( (pdfdx, end_zeros), axis=1 )

    for row in range(1, len(STATES)):
        
        #this corresponds to colz i think
        cutoff = ( len(STATES) - row - pdfdx.shape[1] )
        
        #case 1 far enough from the 'end' of the matrix
        if cutoff >= 0:
            start_zeros = np.zeros((1,row))
            end_zeros = np.zeros((1, len(STATES) - pdfdx.shape[1] - row))
            fmat_new = np.concatenate( (start_zeros, pdfdx, end_zeros), axis=1 )
            fmat0 = np.concatenate((fmat0, fmat_new))
       
        #case 2, too far from the end and need to adjust probs
        else:
            pdf_adj = pdfdx[:,0:cutoff]
            pdf_adj = pdf_adj/pdf_adj.sum(axis=1)
            
            start_zeros = np.zeros((1,row))
            fmat_new = np.concatenate( (start_zeros, pdf_adj), axis=1 )
            fmat0 = np.concatenate((fmat0, fmat_new))
            
    return fmat0

FMAT0 = transition_0(data['replace'],data['miles'],data['miles_next'])

PR_TRANS = FMAT0, FMAT1

In [15]:
def initial_pr(i_obs, x_obs, d=0):
    """initial the probability of view a given state following AM.
    just involves logit to predict
    
    Third arguement involves display"""
    
    X = np.array([x_obs, x_obs**2, x_obs**3]).transpose()
    X = sm.add_constant(X)
    
    model = sm.Logit(i_obs,X)
    fit = model.fit(disp=d)
    if d: print fit.summary()
    
    x_states = np.array([STATES, STATES**2, STATES**3]).transpose()
    x_states = sm.add_constant(x_states)
    
    return fit.predict(x_states)

PR_OBS = initial_pr(data['replace'], data['miles'], d=1)

Optimization terminated successfully.
         Current function value: 0.036201
         Iterations 23
                           Logit Regression Results                           
Dep. Variable:                replace   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8152
Method:                           MLE   Df Model:                            3
Date:                Tue, 15 Jan 2019   Pseudo R-squ.:                  0.1671
Time:                        14:20:03   Log-Likelihood:                -295.26
converged:                       True   LL-Null:                       -354.51
                                        LLR p-value:                 1.623e-25
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.3136      4.188     -4.134      0.000     -25.522      -9.105
x1           149.3089     56

## Step 3: Alternative Value Function Representation

Now, that we have non-parametric estimates of the transition matrix and and choice probabilities we can calculate the value function to learn the importance of mileage to the bus manager.

First let's define some variables:

* Let $F(i)$ be the transition matrix between states depending on the replacement decision $i$. We learned this above using the Guassian kernel.

* Let $P$ be the probability of replacing the engine conditioanl on the mileage. We learned this above using the logit.

* Let $u(i,x;\theta)$ be the cost function.

* Implicitly, there is uncertainity $\epsilon$ each period. We give this the parametric extereme value distribution.

Using just these objects, the model parameters (which we are trying to learn), we can write the value function below

$$V = [I_m - \beta[(1-P) \otimes F(0) + P \otimes F(1)] ]^{-1} [(1-P)*(u(0,x;\theta) + \gamma -ln(1-P)) + P*( u(1,x;\theta) + \gamma -ln(P) ) ]$$

This corresponds to equation (8) in Aguirregabiria Mira 2002. You can find a formal derivation of this in Hotz Miller 1993 and Aguirregabira Mira's paper.

For the purposes of clarifying some symbols:

* $\otimes$ is the Kroenecker product (i.e. column wise). I implemented this by tiling the vector.
* And, $*$ is the Hadamard produce (i.e. element wise)


Also note, I parameterized the cost function using a python `lambda` expression. For those unfamilar, I am passing another function into the routine for calculating the value funciton.

In [34]:
def hm_value(params, cost, pr_obs, pr_trans):
    """calculate value function using hotz miller approach"""
    
    #set up matrices, transition is deterministic
    trans0, trans1 = pr_trans
    
    #calculate value function for all state
    pr_tile = np.tile( pr_obs.reshape( len(STATES) ,1), (1, len(STATES) ))
    
    denom = (np.identity( len(STATES) ) - BETA*(1-pr_tile)*trans0 - BETA*pr_tile*trans1)
    
    numer = ( (1-pr_obs)*(cost(params, STATES, 0) + GAMMA - np.log(1-pr_obs)) + 
                 pr_obs*(cost(params, STATES, 1) + GAMMA - np.log(pr_obs) ) )
    
    value = np.linalg.inv(denom).dot(numer)
    return value

## Step 4: (Psuedo) Maximum Likelihood Estimaton

With the value function we just calculated, we can learn a 'psuedo' likehood of an observation. If we think agents have discount factor $\beta >0$ the true likelihood of each observation would involve calculating the value function each time we calculate the likelihood. Rust actually calculates this likelihood each time using value function iteration.

You might be wondering why recalculate the choice probabilities? Well, now the choice probabilities depend on the parameters we wanted to learn. Ultimately, we wanted to learn how much mileage influences our decision for the future. 

Even though we have the probability of replacement from the data. We recalculate it using the value function. Instead of recalculating the value function on each call to the likelihood function we are just adusting our initial estimates of the likelihood. It turns out that these approaches are both assymptoically consistent.


The formula for the likelihod based on the value function is


This is very similiar to the logit we used to estimate the choice probabilities before. Now, the mileage is influenced by our value function.

 $\Psi$ in Aguirregabiria Mira 2002. 

Aguirregabiria Mira parametrize $\Psi$ using the extreme value distribution below Proposition 3 in their paper.

In [ ]:
def hm_prob(params, cost, pr_obs, pr_trans):
    """calculate kappa (i.e. CCP likelihood) using value function"""
    
    value = hm_value(params, cost, pr_obs, pr_trans)
    value = value - value.min() #subtract out smallest value
    trans0, trans1 = pr_trans
    
    delta1 = np.exp( cost(params, STATES, 1) + BETA*trans1.dot(value))
    delta0 = np.exp( cost(params, STATES, 0) + BETA*trans0.dot(value) )
    
    return delta1/(delta1+delta0)

In [28]:
class CCP(GenericLikelihoodModel):
    """class for estimating the values of R and theta
    using the CCP routine and the helper functions
    above"""
    
    def __init__(self, i, x, x_next, params, cost, **kwds):
        """initialize the class
        
        i - replacement decisions
        x - miles
        x_next - next periods miles
        params - names for cost function parameters
        cost - cost function specification, takes agruements (params, x, i) """
        
        super(CCP, self).__init__(i, x, **kwds)
        
        #data
        self.endog = i #these names don't work exactly
        self.exog = x #the idea is that x is mean indep of epsilon
        self.x_next = x_next
        
        #transitions
        self.pr_obs = initial_pr(i, x)
        self.trans =  transition_0(i,x,x_next), transition_1(i,x,x_next)
        
        #should probably make these class parameters
        self.num_states = ( x.max()/STEP).astype(int) + 2
        self.states = np.arange(x.min(),x.max() + STEP, STEP)
        
        #initial model fit
        self.cost = cost
        self.num_params = len(params)
        self.data.xnames =  params
        self.results = self.fit( start_params=np.ones(self.num_params) )
        
        
    def nloglikeobs(self, params, v=False):
        """psuedo log likelihood function for the CCP estimator"""
        
        # Input our data into the model
        i = self.endog
        x = (self.exog/STEP).astype(int)*STEP #discretized x
           
        #set up hm state pr
        prob = hm_prob(params, self.cost, self.pr_obs, self.trans).transpose()
        prob = interp1d(self.states, prob)
        prob = prob(x)
        
        log_likelihood = (1-i)*np.log(1-prob) + i*np.log(prob)
        
        return -log_likelihood.sum()
    
    
    def iterate(self, numiter):
        """iterate the Hotz Miller estimation procedure 'numiter' times"""
        i = 0
        while(i < numiter):
            #update pr_obs based on parameters
            self.pr_obs = hm_prob(self.results.params, self.cost, self.pr_obs, self.trans)
            
            #refit the model
            self.results = self.fit(start_params=np.ones(self.num_params))
            i = i +1

### Linear Costs

In [31]:
#define cost functon using lambda expression
LINEAR_COST = lambda params, x, i: (1-i)*x*params[i] + i*params[i]

model_ccp = CCP(data['replace'], data['miles'], data['miles_next'], ['theta1','theta2'], LINEAR_COST)
print model_ccp.results.summary()

Optimization terminated successfully.
         Current function value: 0.036544
         Iterations: 63
         Function evaluations: 120
                                 CCP Results                                  
Dep. Variable:                replace   Log-Likelihood:                -298.05
Model:                            CCP   AIC:                             598.1
Method:            Maximum Likelihood   BIC:                             605.1
Date:                Tue, 15 Jan 2019                                         
Time:                        14:37:02                                         
No. Observations:                8156                                         
Df Residuals:                    8155                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

### Quadratic Costs

We can see that the change in specification does not drastically change the estimates. Considering the limited data, the cost function is probability not identified.

In [27]:
QUAD_COST = lambda params, x, i: (1-i)*(x*params[0] + x**2*params[1]) + i*params[2]

model_ccp = CCP(data['replace'], data['miles'], data['miles_next'], ['theta1','theta2', 'theta3'], QUAD_COST)
print model_ccp.results.summary()

Optimization terminated successfully.
         Current function value: 0.036261
         Iterations: 147
         Function evaluations: 260
                                 CCP Results                                  
Dep. Variable:                replace   Log-Likelihood:                -295.75
Model:                            CCP   AIC:                             593.5
Method:            Maximum Likelihood   BIC:                             600.5
Date:                Tue, 15 Jan 2019                                         
Time:                        14:35:24                                         
No. Observations:                8156                                         
Df Residuals:                    8155                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

## Step 5: Iterating the Model

It turns out that you can iterate upon these estimates to converge to the true policy function in Rust 1987. I iterate the value function below. For more information on this you can see.

Victor Aguirregabiria and Pedro Mira's website:
http://individual.utoronto.ca/vaguirre/wpapers/program_code_survey_joe_2008.html

Victor Aguirregabiria and Pedro Mira's 2002 paper
https://www.jstor.org/stable/3082006


In [30]:
model_ccp = CCP(data['replace'], data['miles'], data['miles_next'], ['theta1','theta2'], LINEAR_COST)
model_ccp.iterate(2)
print model_ccp.results.summary()

Optimization terminated successfully.
         Current function value: 0.036544
         Iterations: 63
         Function evaluations: 120
Optimization terminated successfully.
         Current function value: 0.036530
         Iterations: 62
         Function evaluations: 117
Optimization terminated successfully.
         Current function value: 0.036528
         Iterations: 63
         Function evaluations: 118
                                 CCP Results                                  
Dep. Variable:                replace   Log-Likelihood:                -297.93
Model:                            CCP   AIC:                             597.9
Method:            Maximum Likelihood   BIC:                             604.9
Date:                Tue, 15 Jan 2019                                         
Time:                        14:35:56                                         
No. Observations:                8156                                         
Df Residuals:                 